In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/MyDrive/ResearchProject")

Mounted at /content/drive/


In [ ]:
from irp_functions import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

from scipy.stats import ks_2samp
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, f1_score 
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from sklearn.neighbors import NearestNeighbors

In [ ]:
root_path = '/content/drive/My Drive/ResearchProject/' 
dataset_path = root_path + 'Datasets/'
abalone_filename = root_path + 'Datasets/abalone.csv'
df = pd.read_csv(abalone_filename)

In [ ]:
df["Age"] = df["Rings"] + 1.5
df.drop("Rings", axis=1, inplace=True)

df['Male'] = np.where(df['Sex'] == "M", 1, 0)
df['Female'] = np.where(df['Sex'] == "F", 1, 0)
df.drop('Sex', axis=1, inplace=True)

In [ ]:
df = df[df['Age'].isin([10.5, 19.5])]
df['Minority'] = np.where(df['Age'] == 19.5, 1, 0)
df.drop('Age', axis=1, inplace=True)
df

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Male,Female,Minority
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,0,1,0
8,0.475,0.370,0.125,0.5095,0.2165,0.1125,0.1650,1,0,0
19,0.450,0.320,0.100,0.3810,0.1705,0.0750,0.1150,1,0,0
23,0.550,0.415,0.135,0.7635,0.3180,0.2100,0.2000,0,1,0
32,0.665,0.525,0.165,1.3380,0.5515,0.3575,0.3500,1,0,1
...,...,...,...,...,...,...,...,...,...,...
4134,0.595,0.455,0.140,0.9140,0.3895,0.2225,0.2710,0,1,0
4136,0.615,0.495,0.155,1.0805,0.5200,0.1900,0.3200,0,1,0
4159,0.560,0.440,0.135,0.8025,0.3500,0.1615,0.2590,0,1,0
4167,0.500,0.380,0.125,0.5770,0.2690,0.1265,0.1535,1,0,0


In [ ]:
y = df['Minority'].to_numpy()
X = df.drop(['Minority'], axis=1).to_numpy()

In [ ]:
benchmark = ten_experiments(X=X, y=y)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

In [ ]:
benchmark

{1: [{'ROCAUC': 0.9405797101449276, 'confusion_matrix': array([[69,  0],
          [ 4,  1]]), 'f1': 0.33333333333333337, 'precision': 1.0, 'recall': 0.2},
  {'ROCAUC': 0.9710144927536232, 'confusion_matrix': array([[69,  0],
          [ 2,  2]]), 'f1': 0.6666666666666666, 'precision': 1.0, 'recall': 0.5},
  {'ROCAUC': 0.8858695652173914, 'confusion_matrix': array([[69,  0],
          [ 4,  0]]), 'f1': 0.0, 'precision': 0.0, 'recall': 0.0},
  {'ROCAUC': 0.5253623188405797, 'confusion_matrix': array([[69,  0],
          [ 4,  0]]), 'f1': 0.0, 'precision': 0.0, 'recall': 0.0},
  {'ROCAUC': 0.9818840579710145, 'confusion_matrix': array([[69,  0],
          [ 4,  0]]), 'f1': 0.0, 'precision': 0.0, 'recall': 0.0},
  {'ROCAUC': 0.6141304347826086, 'confusion_matrix': array([[68,  1],
          [ 3,  1]]), 'f1': 0.3333333333333333, 'precision': 0.5, 'recall': 0.25},
  {'ROCAUC': 0.7807971014492754, 'confusion_matrix': array([[68,  1],
          [ 2,  2]]), 'f1': 0.5714285714285715, 'precision

In [ ]:
benchmark_data = get_DataFrame_from_results(benchmark)
benchmark_data

,TN,FN,TP,FP,ROCAUC,f1,precision,recall
0,69.0,4.0,1.0,0.0,0.940580,0.333333,1.0,0.20
1,69.0,2.0,2.0,0.0,0.971014,0.666667,1.0,0.50
2,69.0,4.0,0.0,0.0,0.885870,0.000000,0.0,0.00
3,69.0,4.0,0.0,0.0,0.525362,0.000000,0.0,0.00
4,69.0,4.0,0.0,0.0,0.981884,0.000000,0.0,0.00
...,...,...,...,...,...,...,...,...
95,69.0,3.0,1.0,0.0,0.719203,0.400000,1.0,0.25
96,68.0,3.0,1.0,1.0,0.702899,0.333333,0.5,0.25
97,69.0,3.0,1.0,0.0,0.706522,0.400000,1.0,0.25
98,69.0,1.0,3.0,0.0,1.000000,0.857143,1.0,0.75


In [ ]:
SMOTE = {}
for oversampling_rate in range(100, 1100, 100):
  results = ten_experiments(X=X, y=y, over_sampler="SMOTE", N=oversampling_rate)
  data = get_DataFrame_from_results(results)
  SMOTE[oversampling_rate] = data

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

In [ ]:
Outlier_SMOTE_Euclidean = {}
for oversampling_rate in range(100, 1100, 100):
  results = ten_experiments(X=X, y=y, over_sampler="Outlier_SMOTE", 
                            N=oversampling_rate)
  data = get_DataFrame_from_results(results)
  Outlier_SMOTE_Euclidean[oversampling_rate] = data

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

In [ ]:
Outlier_SMOTE_Manhattan = {}
for oversampling_rate in range(100, 1100, 100):
  results = ten_experiments(X=X, y=y, over_sampler="Outlier_SMOTE", 
                            metric="manhattan" ,N=oversampling_rate)
  data = get_DataFrame_from_results(results)
  Outlier_SMOTE_Manhattan[oversampling_rate] = data

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

In [ ]:
Outlier_SMOTE_Chebyshev = {}
for oversampling_rate in range(100, 1100, 100):
  results = ten_experiments(X=X, y=y, over_sampler="Outlier_SMOTE", 
                            metric="chebyshev" ,N=oversampling_rate)
  data = get_DataFrame_from_results(results)
  Outlier_SMOTE_Chebyshev[oversampling_rate] = data

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

In [ ]:
os.chdir("/content/drive/MyDrive/ResearchProject/Results/Abalone")

In [ ]:
benchmark_data.to_csv('benchmark.csv')

In [ ]:
for oversampling_rate in range(100, 1100, 100):
  prefix = str(oversampling_rate)
  
  smote = SMOTE[oversampling_rate]
  euclidean = Outlier_SMOTE_Euclidean[oversampling_rate]
  manhattan = Outlier_SMOTE_Manhattan[oversampling_rate]
  chebyshev = Outlier_SMOTE_Chebyshev[oversampling_rate]

  smote_name = prefix + "_SMOTE.csv"
  euclidean_name = prefix + "_Euclidean_Outlier_SMOTE.csv"
  manhattan_name = prefix + "_Manhattan_Outlier_SMOTE.csv"
  chebyshev_name = prefix + "_Chebyshev_Outlier_SMOTE.csv"

  smote.to_csv(smote_name)
  euclidean.to_csv(euclidean_name)
  manhattan.to_csv(manhattan_name)
  chebyshev.to_csv(chebyshev_name)  